In [ ]:
from torch.utils.data import DataLoader
from torchvision.models import get_model, ViT_B_32_Weights
import pytorch_lightning as pl
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import import_ipynb
from vit import UAV_vit
from dataset import BaseDataset

In [ ]:
CHECKPOINT_PATH = './mlruns/0/c25ee6cf7dca4d15b4868e4a344dd807/artifacts/restored_model_checkpoint/uav_vit-checkpoint.ckpt'
GROUND_TRUTH_PATH = '../data/ground_truth/corn_plant_height_ground_truth.xlsx'
SAMPLES_PATH = "../data/raw_samples"
IMG_SIZE = (224,224)
BATCH_SIZE = 8

In [ ]:
# Loading the ground truth
df = pd.read_excel(GROUND_TRUTH_PATH)
df[['DataFile 3', 'PHT(m)', 'Elev maximum (m)']].head()
# Ground Truth Preprocessing

def renaming(filename):
    return filename[:-4] + ".png"

def elev_avg(row):
    return (row['PHT(m)'] + row["Elev maximum (m)"]) / 2

# Apply renaming and averaging
df["filename"] = df["DataFile 3"].apply(renaming)
df["elevation_avg"] = (df.apply(elev_avg, axis=1))

# Elevation format conversion to float32
df["elevation_avg"] = df["elevation_avg"].astype(np.float32)
df[['filename', 'elevation_avg']].head()

# Targets normalization

scaler = MinMaxScaler()
df["elevation"] = scaler.fit_transform(df[["elevation_avg"]])

In [ ]:
samplesDataset = BaseDataset(SAMPLES_PATH, IMG_SIZE, normalize=False)
samplesDataloader = DataLoader(samplesDataset, batch_size=BATCH_SIZE)

In [ ]:
model_path = '../' + CHECKPOINT_PATH
backbone = get_model('vit_b_32', weights=ViT_B_32_Weights)
model = UAV_vit.load_from_checkpoint(model_path, backbone=backbone)
model.eval()


In [ ]:
# Predictions
trainer = pl.Trainer()
predictions = trainer.predict(model, dataloaders=samplesDataloader, return_predictions=True)

In [ ]:
predictions = np.concatenate(predictions)
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1,1))
predictions = [item.item() for sublist in predictions for item in sublist]
print("Predicted results:\n", *predictions, sep='\n')